#### load libraries and define db_connection 

In [1]:
import os
import json
import time
from datetime import datetime, timezone
import psycopg2
from psycopg2.extras import execute_values
import aiosql
import multiprocessing as mp
from dotenv import load_dotenv
from operator import itemgetter

from pgnetworks_processing.python.utilities import Config, create_run_id
from pgnetworks_processing.python import functions

query dml.create_function_public_ghh_decode_id_to_hash at /home/matthiasdaues/Documents/datenschoenheit/pgNetworks/pgnetworks_processing/pgnetworks_processing/sql/dml/ghh_functions.sql:1 may not be a select, consider adding an operator, eg '!'
query dml.drop_function_public_ghh_decode_id_to_hash at /home/matthiasdaues/Documents/datenschoenheit/pgNetworks/pgnetworks_processing/pgnetworks_processing/sql/dml/ghh_functions.sql:35 may not be a select, consider adding an operator, eg '!'
query dml.create_function_public_ghh_decode_hash_to_wkt at /home/matthiasdaues/Documents/datenschoenheit/pgNetworks/pgnetworks_processing/pgnetworks_processing/sql/dml/ghh_functions.sql:41 may not be a select, consider adding an operator, eg '!'
query dml.drop_function_public_ghh_decode_hash_to_wkt at /home/matthiasdaues/Documents/datenschoenheit/pgNetworks/pgnetworks_processing/pgnetworks_processing/sql/dml/ghh_functions.sql:60 may not be a select, consider adding an operator, eg '!'
query dml.create_functi

In [2]:
# set run_id and initiate workstep_idx

run_id = create_run_id()
workstep_idx = 0

print(run_id)

1742329140


#### tear down and set stuff up

In [3]:
# tear stuff down and set it up fresh

with psycopg2.connect(Config.connect_db) as conn:
    try:
#       # drop tables and types
        Config.queries.ddl.drop_table_edges(conn)
        Config.queries.ddl.drop_table_vertex_2_edge(conn)
        Config.queries.ddl.drop_table_junctioned_edges(conn)
        Config.queries.ddl.drop_table_segments(conn)
        Config.queries.ddl.drop_type_segment_processing(conn)
        Config.queries.ddl.drop_table_nodes(conn)
        Config.queries.ddl.drop_table_selector_grid(conn)
#       # Config.queries.ddl.drop_table_log(conn)

#       # drop procedures or functions
        Config.queries.dml.drop_procedure_copy_segments_to_edges(conn)
        Config.queries.dml.drop_procedure_join_vertex_2_edge(conn)
        Config.queries.dml.drop_procedure_process_junctions_and_edges(conn)
        Config.queries.dml.drop_procedure_calculate_selector_grid(conn)
        Config.queries.dml.drop_procedure_segmentize_road_network(conn)
        Config.queries.dml.drop_procedure_count_node_degree(conn)
        
#       # rebuild tables and types
        Config.queries.ddl.create_table_edges(conn)
        Config.queries.ddl.create_table_vertex_2_edge(conn)
        Config.queries.ddl.create_table_junctioned_edges(conn)
        Config.queries.ddl.create_type_segment_processing(conn)
        Config.queries.ddl.create_table_segments(conn)
        Config.queries.ddl.create_table_nodes(conn)
        Config.queries.ddl.create_table_selector_grid(conn)
#       # Config.queries.ddl.create_table_log(conn)
        
#       # create and replace assets
        Config.queries.dml.create_procedure_copy_segments_to_edges(conn)
        Config.queries.dml.create_procedure_join_vertex_2_edge(conn)
        Config.queries.dml.create_procedure_process_junctions_and_edges(conn)
        Config.queries.dml.create_procedure_calculate_selector_grid(conn)
        Config.queries.dml.create_procedure_segmentize_road_network(conn)
        Config.queries.dml.create_procedure_count_node_degree(conn)
        
        conn.commit()
    
    except psycopg2.Error as e:
        print(e)

#### download sources and copy data to DB

#### preprocess data in the DB

##### *perform processing*

In [ ]:
# set lower and upper bounds for the "join_vertex_2_edge" procedure

from pgnetworks_processing.python import functions

chunk_bound_query_name = 'find_bounds_in_poi_table'
workstep_query_name = 'join_vertex_2_edge'
chunk_size = Config.CHUNK_SIZE
workstep_idx += 1
params_list = functions.create_range_bound_params_list(chunk_bound_query_name, workstep_query_name, chunk_size, workstep_idx, run_id)
# params_list = params_list[:1]
# params_list

In [5]:
# join the vertices to the closest edge

workstep_idx += 1
functions.multiprocess_id_range_workstep(params_list, workstep_query_name, workstep_idx, run_id)

In [6]:
# index the table vertex_2_edge for the next bound search run

workstep_idx += 1
functions.create_index('create_index_vertex_2_edge_edge_id_idx', workstep_idx, run_id)

In [7]:
# find the bounds for the "process junctions and edges" procedure

from pgnetworks_processing.python import functions

chunk_bound_query_name = 'find_bounds_in_vertex_2_edge'
workstep_query_name = 'process_junctions_and_edges'
chunk_size = Config.EDGE_PROCESSING_CHUNK_SIZE
workstep_idx += 1
params_list = functions.create_range_bound_params_list(chunk_bound_query_name, workstep_query_name, chunk_size, workstep_idx, run_id)
# params_list = params_list[:1]
# params_list

In [8]:
# process the junctions and edges (segmentize the near_net edges)

workstep_idx += 1
functions.multiprocess_id_range_workstep(params_list, workstep_query_name, workstep_idx, run_id)

In [9]:
# index the table segments for the next bound search run

workstep_idx += 1
functions.create_index('create_index_segments_edge_id_idx', workstep_idx, run_id)

In [16]:
# create the selector grid

workstep_idx += 1
params_list = functions.calculate_selector_grid(workstep_idx, run_id)
# params_list = params_list[:1]

In [3]:
# create the parameter list for the spatially bounded workstep

spatial_bound_query_name = 'select_selector_grid'
spatial_workstep_query_name = 'segmentize_road_network'
workstep_idx += 1
params_list = functions.create_spatial_workstep_params_list(spatial_bound_query_name, spatial_workstep_query_name, workstep_idx, run_id)
# params_list = params_list[:1]
# print(len(params_list))
# params_list

In [5]:
# process the remaining road network edges (process the far_net edges)

workstep_idx += 1
functions.multiprocess_spatial_workstep(params_list, spatial_workstep_query_name, workstep_idx, run_id)

In [6]:
# index the segments table

workstep_idx += 1
functions.create_index('create_index_segments_geom_idx', workstep_idx, run_id)
workstep_idx += 1
functions.create_index('create_index_segments_node_1_idx', workstep_idx, run_id)
workstep_idx += 1
functions.create_index('create_index_segments_node_2_idx', workstep_idx, run_id)

DuplicateTable: relation "segments_geom_idx" already exists


In [7]:
# find the bounds for the "copy segments to edges" procedure

from pgnetworks_processing.python import functions

chunk_bound_query_name = 'find_bounds_in_segments'
workstep_query_name = 'copy_segments_to_edges'
chunk_size = 5000000
workstep_idx += 1
params_list = functions.create_range_bound_params_list(chunk_bound_query_name, workstep_query_name, chunk_size, workstep_idx, run_id)
# params_list = params_list[:1]
# params_list

In [ ]:
# copy segments to edges

workstep_idx += 1
functions.singleprocess_id_range_workstep(params_list, workstep_query_name, workstep_idx, run_id)

In [ ]:
# index the edges table

workstep_idx += 1
functions.create_index('create_index_edges_geom_idx', workstep_idx, run_id)
workstep_idx += 1
functions.create_index('create_index_edges_node_1_id_idx', workstep_idx, run_id)
workstep_idx += 1
functions.create_index('create_index_edges_node_2_id_idx', workstep_idx, run_id)
workstep_idx += 1
functions.create_index('create_index_source_edge_id_idx', workstep_idx, run_id)